In [0]:
%sql
-- Notebook: src/init/00_master_contract
-- Description: Defines the strict schemas for Bronze, Silver, and Gold layers.

-- ==========================================
-- 1. BRONZE LAYER (Raw + Metadata)
-- ==========================================
CREATE TABLE IF NOT EXISTS bronze_schema_holder (
    date DATE NOT NULL,
    symbol STRING NOT NULL,
    ingestion_timestamp TIMESTAMP NOT NULL,
    source_file STRING NOT NULL,
    value STRING NOT NULL, -- Raw JSON payload
    _rescued_data STRING,
    statement_type STRING NOT NULL
) USING DELTA;

-- Constraint: Ensure we only accept valid financial statements
ALTER TABLE bronze_schema_holder 
DROP CONSTRAINT IF EXISTS valid_statement_type;

ALTER TABLE bronze_schema_holder 
ADD CONSTRAINT valid_statement_type 
CHECK (statement_type IN ('income-statement', 'balance-sheet-statement', 'cash-flow-statement'));


-- ==========================================
-- 2. SILVER LAYER (Cleaned Financials)
-- ==========================================

-- A. Income Statement
CREATE TABLE IF NOT EXISTS income_staement_silver_schema_holder (
    date DATE NOT NULL,
    symbol STRING NOT NULL,
    incomeTaxExpense BIGINT NOT NULL,
    interestExpense BIGINT NOT NULL,
    revenue BIGINT NOT NULL,
    costOfRevenue BIGINT NOT NULL,
    grossProfit BIGINT,
    epsDiluted DECIMAL(10, 4),
    operatingExpenses BIGINT,
    ebit BIGINT NOT NULL,
    netIncome BIGINT,
    weightedAverageShsOutDil BIGINT NOT NULL
) USING DELTA;

-- B. Balance Sheet
CREATE TABLE IF NOT EXISTS balance_sheet_staement_silver_schema_holder (
    date DATE NOT NULL,
    symbol STRING NOT NULL,
    cashAndCashEquivalents BIGINT NOT NULL,
    netReceivables BIGINT NOT NULL,
    totalCurrentLiabilities BIGINT NOT NULL,
    totalCurrentAssets BIGINT NOT NULL,
    totalAssets BIGINT NOT NULL,   
    accountPayables BIGINT NOT NULL,
    inventory BIGINT NOT NULL,
    totalDebt BIGINT NOT NULL,
    netDebt BIGINT,
    totalEquity BIGINT NOT NULL
) USING DELTA;

-- C. Cash Flow Statement
CREATE TABLE IF NOT EXISTS cash_flow_staement_silver_schema_holder (
    date DATE NOT NULL,
    symbol STRING NOT NULL,
    depreciationAndAmortization BIGINT NOT NULL,
    operatingCashFlow BIGINT NOT NULL,
    capitalExpenditure BIGINT NOT NULL,
    freeCashFlow BIGINT NOT NULL,
    netDividendsPaid BIGINT NOT NULL
) USING DELTA;


-- ==========================================
-- 3. GOLD LAYER (Aggregated Metrics)
-- ==========================================
CREATE TABLE IF NOT EXISTS gold_financial_statement_schema_holder (
    date DATE NOT NULL,
    symbol STRING NOT NULL,
    
    -- Calculated Metrics
    nopat BIGINT,
    ebit BIGINT,
    gross_margin DECIMAL(10, 4),
    depreciation BIGINT,
    working_capital BIGINT,
    delta_wc BIGINT,
    capex BIGINT,
    reinvestment_rate DECIMAL(10, 4),
    net_debt BIGINT,
    liquidity_ratio DECIMAL(10, 4),
    interest_coverage_ratio DECIMAL(10, 4),
    calculated_fcf BIGINT,
    
    -- Core Financials (Persisted for Context)
    total_debt DECIMAL(30, 4),
    tax_expense DECIMAL(30, 4),
    interest_expense DECIMAL(30, 4),
    shares_outstanding DECIMAL(30, 4),

    -- Primary Key Constraint (Composite)
    CONSTRAINT pk_gold_financials PRIMARY KEY (date, symbol)
) USING DELTA;